In [11]:
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from data_preprocess import data_preprocess
from feature_extractor import FeatureExtractor
from test_model import test_model

In [2]:
# Load the data and extract gesture segments from the data.
data_path = 'Project_Data_EE4C12_S&S_EMG.csv'
gesture_windows = data_preprocess(data_path)

In [3]:
# Extract features for each gesture
feature_extractor = FeatureExtractor()
features = feature_extractor.extract_features(gesture_windows)

Analyze the class balances using histogram of class samples

In [ ]:
class_balances = features['gesture'].value_counts()
print(class_balances)
plt.figure(figsize=(10, 6))
class_balances.plot(kind='bar', color='skyblue')
plt.title('Class Distribution')
plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.xticks(rotation=0)
plt.show()

It shows that there are too few samples (8 samples) from class 7. We can give that class a higher weight in the loss functions of the future models but even training might not be good enough with this many samples. So we decided to remove the class 7 from the data.

Also, unmarked data belonging to class 0 dominates other classes in terms of number of samples and generally classifiers work better when all classes are balanced. So we can give it a lower weight or subsample the class 0. We chose to subsample the class.

In [ ]:
# Remove class 7 from the features dataframe
features_filtered = features[features['gesture'] != 7]

# Get the number of samples for the smallest class (excluding class 0)
min_class_count = features_filtered['gesture'].value_counts().min()

# Subsample class 0 to have the same number of samples as the smallest class
class_0_indices = features_filtered[features_filtered['gesture'] == 0].index
subsampled_class_0_indices = np.random.choice(class_0_indices, min_class_count, replace=False)

# Get the indices of all other classes
other_class_indices = features_filtered[features_filtered['gesture'] != 0].index

# Combine the subsampled class 0 indices with the other class indices
new_indices = np.concatenate([subsampled_class_0_indices, other_class_indices])

# Create the new features dataframe
features_balanced = features_filtered.loc[new_indices]

# Display the class distribution of the new features dataframe
plt.figure(figsize=(10, 6))
features_balanced['gesture'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Balanced Class Distrubution')
plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Create train-test split
X_train, X_test, y_train, y_test = train_test_split(features_balanced.drop(columns=['gesture']), features_balanced['gesture'], test_size=0.15, random_state=42)

# Reset indices of the resulting datasets
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

# Create train2-validation split
X_train2, X_validation, y_train2, y_validation = train_test_split(X_train, y_train, test_size=0.30, random_state=42)

X_train2.reset_index(drop=True, inplace=True)
y_train2.reset_index(drop=True, inplace=True)
X_validation.reset_index(drop=True, inplace=True)
y_validation.reset_index(drop=True, inplace=True)

# Display the shapes of the resulting datasets
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_train shape: {X_train2.shape}")
print(f"y_train shape: {y_train2.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")
print(f"X_validation shape: {X_validation.shape}")
print(f"y_validation shape: {y_validation.shape}")


In [7]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Use the same scaler to transform other splits
X_test_scaled = scaler.transform(X_test)
X_train2_scaled = scaler.transform(X_train2)
X_validation_scaled = scaler.transform(X_validation)

# Convert the scaled data back to DataFrame for better readability
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)
X_train2_scaled = pd.DataFrame(X_train2_scaled, columns=X_train2.columns)
X_validation_scaled = pd.DataFrame(X_validation_scaled, columns=X_validation.columns)

Find the best C value for the Logistic Regression model based on accuracy on validation set. Then train the best model using whole train set and test it.

In [ ]:
C_values = [0.001, 0.01, 0.1, 1, 10]
max_min_f1_score = 0
best_c_value = None

for c_val in C_values:
    logistic_model = LogisticRegression(C=c_val, max_iter=1000, multi_class="multinomial",
                                        tol=1e-4, random_state=42)

    # Train the model using the scaled training data (not containing validation set samples)
    logistic_model.fit(X_train2_scaled, y_train2)

    # Print the training accuracy
    train_accuracy = logistic_model.score(X_train2_scaled, y_train2)

    # Validate model using the scaled validation data
    metrics = test_model(logistic_model, X_validation_scaled, y_validation)
    print(f"LR model with c={c_val} reached {metrics['min_f1_score']:.2f} minimum f1 score on the validation set.")

    if (max_min_f1_score < metrics['min_f1_score']):
        max_min_f1_score = metrics['min_f1_score']
        best_c_value = c_val

# Train the best model with whole training set
best_model = LogisticRegression(C=best_c_value, max_iter=1000, multi_class="multinomial",
                                tol=1e-4, random_state=42)
best_model.fit(X_train_scaled, y_train)

# Test the best model using the scaled test data
metrics_lr = test_model(best_model, X_test_scaled, y_test, verbose=True)


We tune regularization parameter alpha of Ridge Classifier using the validation set. We chose the alpha value with highest accuracy on validation set. Then we trained that model using the whole training set and tested it.

In [ ]:
alpha_vals = [1e-5, 1e-4, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
max_min_f1_score = 0
best_alpha_value = None
for alpha_val in alpha_vals:
    model = RidgeClassifier(alpha=alpha_val, max_iter=1000,
                                        tol=1e-4, random_state=42)

    # Train the model using the scaled training data (not containing validation set samples)
    model.fit(X_train2_scaled, y_train2)

    # Print the training accuracy
    train_accuracy = model.score(X_train2_scaled, y_train2)

    # Validate model using the scaled validation data
    metrics = test_model(model, X_validation_scaled, y_validation)
    print(f"Ridge Classifier model with alpha={alpha_val} reached {metrics['min_f1_score']:.2f} minimum f1 score on the validation set.")

    if (max_min_f1_score < metrics['min_f1_score']):
        max_min_f1_score = metrics['min_f1_score']
        best_alpha_value = alpha_val

# Train the best model with whole training set
highest_accuracy_model = RidgeClassifier(alpha=best_alpha_value, max_iter=1000,
                                        tol=1e-4, random_state=42)
highest_accuracy_model.fit(X_train_scaled, y_train)

# Test the best model using the scaled test data
metrics_ridge_classifier = test_model(highest_accuracy_model, X_test_scaled, y_test, verbose=True)

Hyperparameters of an MLP are as follows:
1. Number of hidden layers
2. Hidden layer dimensions
3. Activation functions
4. LR scheduling

We tuned LR scheduling using the built-in functionality of MLP using 10% of its training data as a validation set. However, we kept using our own validation set to tune the rest of the hyperparameters using a grid search strategy.

In [ ]:
n_hidden_layers = [1, 2, 3, 4]
hidden_layer_sizes = [(100,), (200,), (300,)]
activation_functions = ['relu', 'tanh', 'logistic']
highest_min_f1_score = 0
highest_accuracy_model_params = {}

for n_layers in n_hidden_layers:
    for layer_size in hidden_layer_sizes:
        for activation in activation_functions:
            layer_size = (layer_size[0],) * n_layers
            mlp_model = MLPClassifier(hidden_layer_sizes=layer_size, max_iter=1000, random_state=42,
                                       activation=activation, learning_rate='adaptive',
                                       learning_rate_init=1e-3, tol=1e-4)

            # Train the model using the scaled training data
            mlp_model.fit(X_train2_scaled, y_train2)
            
            # Validate model using the scaled validation data
            metrics = test_model(mlp_model, X_validation_scaled, y_validation)
            min_f1_score = metrics['min_f1_score']
            print(f"MLP model with {n_layers} hidden layers, {layer_size} neurons, and {activation} activation function reached {min_f1_score:.2f} minimum f1 score on the validation set.")

            if (highest_min_f1_score < min_f1_score):
                highest_min_f1_score = min_f1_score
                highest_accuracy_model_params['n_hidden_layers'] = n_layers
                highest_accuracy_model_params['hidden_layer_sizes'] = layer_size
                highest_accuracy_model_params['activation'] = activation         

print(highest_accuracy_model_params)

# Train the best model using the scaled training data
best_model = MLPClassifier(hidden_layer_sizes=highest_accuracy_model_params['hidden_layer_sizes'],
                            max_iter=1000, random_state=42, activation=highest_accuracy_model_params['activation'],
                              learning_rate='adaptive', learning_rate_init=1e-3, tol=1e-4)

best_model.fit(X_train_scaled, y_train)

# Print the training accuracy
train_accuracy_mlp = best_model.score(X_train_scaled, y_train)

metrics_mlp = test_model(best_model, X_test_scaled, y_test, True)



In [ ]:
# Assuming X_train_scaled is the data we are applying PCA on
# Step 1: Apply PCA to the training data and fit the model
pca = PCA(n_components=20)
pca.fit(X_train_scaled)  # You can apply to X_train, X_test, or the full dataset

# Step 2: Calculate explained variance and cumulative variance
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

# Step 3: Plot the explained variance ratio and cumulative explained variance ratio
plt.figure(figsize=(8, 5))

# Plot the explained variance ratio for each component
plt.plot(range(1, 21), explained_variance_ratio, 'o-', label='Explained Variance Ratio', color='b')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance Ratio and Cumulative Explained Variance')
plt.xticks(range(1, 21))

# Plot the cumulative explained variance ratio
plt.plot(range(1, 21), cumulative_explained_variance, 'o-', label='Cumulative Explained Variance', color='r')
plt.legend()

plt.grid(True)
plt.show()

# Step 4: Now reduce dimensionality to 2 components for visualization
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_train_scaled)  # The data is now reduced to 2 dimensions

# Step 5: Plot the first two principal components
plt.figure(figsize=(8, 5))

# Assuming y_train is the label and we have classes 0 to 3 (adjust labels/colors as per your data)
plt.scatter(X_pca_2d[y_train == 0, 0], X_pca_2d[y_train == 0, 1], alpha=0.7, edgecolors='w', color='purple', s=50, label='Class 0')
plt.scatter(X_pca_2d[y_train == 1, 0], X_pca_2d[y_train == 1, 1], alpha=0.7, edgecolors='w', color='yellow', s=50, label='Class 1')
plt.scatter(X_pca_2d[y_train == 2, 0], X_pca_2d[y_train == 2, 1], alpha=0.7, edgecolors='w', color='black', s=50, label='Class 2')
plt.scatter(X_pca_2d[y_train == 3, 0], X_pca_2d[y_train == 3, 1], alpha=0.7, edgecolors='w', color='blue', s=50, label='Class 3')

plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('First Two Principal Components of PCA')
plt.legend()
plt.show()


In [ ]:
from sklearn.cluster import KMeans
from sklearn.svm import SVC


# Initialize the K-Means model
kmeans_model = KMeans(n_clusters=len(set(y_train)), random_state=42)  # Adjust n_clusters based on your dataset

# Fit the K-Means model using the training data
kmeans_model.fit(X_train_scaled)

# Predict cluster labels for training, test, and validation datasets
train_clusters = kmeans_model.predict(X_train_scaled)
test_clusters = kmeans_model.predict(X_test_scaled)
validation_clusters = kmeans_model.predict(X_validation_scaled)

# You can check inertia (within-cluster sum of squares) for evaluating K-Means
print(f"K-Means Inertia: {kmeans_model.inertia_:.4f}")

# Confusion matrix for K-Means (test set)
conf_matrix_kmeans_test = confusion_matrix(y_test, test_clusters)
print("\nK-Means Test Confusion Matrix:\n", conf_matrix_kmeans_test)

# Confusion matrix for K-Means (validation set)
conf_matrix_kmeans_validation = confusion_matrix(y_validation, validation_clusters)
print("\nK-Means Validation Confusion Matrix:\n", conf_matrix_kmeans_validation)

# Calculate the accuracy for K-Means clustering by comparing the clusters with true labels
accuracy_kmeans_test = accuracy_score(y_test, test_clusters)
accuracy_kmeans_validation = accuracy_score(y_validation, validation_clusters)

print(f"K-Means Test Accuracy: {accuracy_kmeans_test:.4f}")
print(f"K-Means Validation Accuracy: {accuracy_kmeans_validation:.4f}")



In [ ]:
# Initialize the SVM model
svm_model = SVC(kernel='linear', C=0.1, random_state=42)  # You can adjust the kernel and C parameter as needed

# Train the SVM model
svm_model.fit(X_train2_scaled, y_train2)

# Evaluate the SVM model on training, test, and validation datasets
train_accuracy_svm = svm_model.score(X_train2_scaled, y_train2)
test_accuracy_svm = svm_model.score(X_test_scaled, y_test)
validation_accuracy_svm = svm_model.score(X_validation_scaled, y_validation)

# Print training, test, and validation accuracy for SVM
print(f"SVM Training Accuracy: {train_accuracy_svm:.4f}")
print(f"SVM Test Accuracy: {test_accuracy_svm:.4f}")
print(f"SVM Validation Accuracy: {validation_accuracy_svm:.4f}")

# Generate confusion matrix and classification report for SVM on test and validation datasets
test_predictions_svm = svm_model.predict(X_test_scaled)
validation_predictions_svm = svm_model.predict(X_validation_scaled)

conf_matrix_svm_test = confusion_matrix(y_test, test_predictions_svm)
class_report_svm_test = classification_report(y_test, test_predictions_svm)
print("\nSVM Test Confusion Matrix:\n", conf_matrix_svm_test)
print("\nSVM Test Classification Report:\n", class_report_svm_test)

conf_matrix_svm_validation = confusion_matrix(y_validation, validation_predictions_svm)
class_report_svm_validation = classification_report(y_validation, validation_predictions_svm)
print("\nSVM Validation Confusion Matrix:\n", conf_matrix_svm_validation)
print("\nSVM Validation Classification Report:\n", class_report_svm_validation)

In [ ]:

# Step 1: Apply PCA to reduce the dataset to 10 components
pca_10 = PCA(n_components=10)
X_train_pca = pca_10.fit_transform(X_train_scaled)  # Reduce training data
X_test_pca = pca_10.transform(X_test_scaled)        # Apply the same transformation to test data
X_validation_pca = pca_10.transform(X_validation_scaled)  # Apply transformation to validation data

# Step 2: Train the SVM model using the reduced dataset (10 PCA components)
svm_model_pca = SVC(kernel='linear', C=1, random_state=42)
svm_model_pca.fit(X_train_pca, y_train)

# Step 3: Train the K-Means Clustering model using the reduced dataset (10 PCA components)
kmeans_model_pca = KMeans(n_clusters=len(set(y_train)), random_state=42)  # Number of clusters = number of classes
kmeans_model_pca.fit(X_train_pca)

# Step 4: Evaluate the SVM model
# Accuracy on training, test, and validation sets
train_accuracy_pca = svm_model_pca.score(X_train_pca, y_train)
test_accuracy_pca = svm_model_pca.score(X_test_pca, y_test)
validation_accuracy_pca = svm_model_pca.score(X_validation_pca, y_validation)

print(f"SVM with PCA (10 components) Training Accuracy: {train_accuracy_pca:.4f}")
print(f"SVM with PCA (10 components) Test Accuracy: {test_accuracy_pca:.4f}")
print(f"SVM with PCA (10 components) Validation Accuracy: {validation_accuracy_pca:.4f}")

# Step 5: Generate confusion matrix and classification report for the SVM model on test and validation sets
test_predictions_pca = svm_model_pca.predict(X_test_pca)
conf_matrix_pca_test = confusion_matrix(y_test, test_predictions_pca)
class_report_pca_test = classification_report(y_test, test_predictions_pca)

print("\nSVM with PCA Test Confusion Matrix:\n", conf_matrix_pca_test)
print("\nSVM with PCA Test Classification Report:\n", class_report_pca_test)

validation_predictions_pca = svm_model_pca.predict(X_validation_pca)
conf_matrix_pca_validation = confusion_matrix(y_validation, validation_predictions_pca)
class_report_pca_validation = classification_report(y_validation, validation_predictions_pca)

print("\nSVM with PCA Validation Confusion Matrix:\n", conf_matrix_pca_validation)
print("\nSVM with PCA Validation Classification Report:\n", class_report_pca_validation)

# Step 6: Evaluate K-Means Clustering
# Predict clusters for the test and validation datasets
test_clusters_pca = kmeans_model_pca.predict(X_test_pca)
validation_clusters_pca = kmeans_model_pca.predict(X_validation_pca)

# Confusion matrix for K-Means (test set)
conf_matrix_kmeans_test = confusion_matrix(y_test, test_clusters_pca)
print("\nK-Means with PCA Test Confusion Matrix:\n", conf_matrix_kmeans_test)

# Confusion matrix for K-Means (validation set)
conf_matrix_kmeans_validation = confusion_matrix(y_validation, validation_clusters_pca)
print("\nK-Means with PCA Validation Confusion Matrix:\n", conf_matrix_kmeans_validation)

# Calculate the accuracy for K-Means clustering (by comparing clusters to labels)
accuracy_kmeans_test = accuracy_score(y_test, test_clusters_pca)
accuracy_kmeans_validation = accuracy_score(y_validation, validation_clusters_pca)

print(f"K-Means with PCA Test Accuracy: {accuracy_kmeans_test:.4f}")
print(f"K-Means with PCA Validation Accuracy: {accuracy_kmeans_validation:.4f}")

# Step 7: Plot the explained variance ratio and cumulative variance (optional)
explained_variance_ratio = pca_10.explained_variance_ratio_
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

plt.figure(figsize=(8, 5))
plt.plot(range(1, 11), explained_variance_ratio, 'o-', label='Explained Variance Ratio', color='b')
plt.plot(range(1, 11), cumulative_explained_variance, 'o-', label='Cumulative Explained Variance', color='r')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance Ratio and Cumulative Explained Variance (10 Components)')
plt.legend()
plt.grid(True)
plt.show()
